# Lesson XX:
# Introduction to GridAPPS-D Topics

This tutorial introduces the topics library in GridAPPS-D and how it can be used

__Learning Objectives:__

At the end of the tutorial, the user should be able to

* Describe what are GridAPPS-D Topics
* Explain the difference between `/queue/` and `/topic/` channels
* Import and use the GridAPPSD-Python library of topics
* Implement shortcut functions for creating GridAPPS-D Topics

---
# What are Topics in GridAPPS-D?

When communicating with the GridAPPS-D Platform through API, it is necessary to specify a topic, which tells (GOSS Message Bus??) on which channel to communicate with the application.

In the previous lessons, we manually specified the topic as a string, like this

`topic = "goss.gridappsd.process.request.data.powergridmodel"`

It is entirely possible to use this method throughout your application. However, this requires you to look up or remember the correct topic for each API call.

---
# `/queue/` vs `/topic/`

GridAPPS-D uses two types of topics to determine the visibility of the API call to other applications and services.

## Queue Channels

`/queue/` is used for communication channels where only the GridAPPS-D Platform is listening to the API call. These API calls are processed on a first-in, first-out basis. There is only one subscriber to the communication channel.

API calls to the Blazegraph database, Logs, Timeseries database, Config files, and Platform status are all queue channels. All the GridAPPS-D Topics for queue channels typically do not change over the course of an application or simulation run.

In the GridAPPSD-Python library, it is assumed that a topic is a queue channel if not otherwise specified. These two GridAPPS-D Topic definitions are equivalent:

`topic = '/queue/goss.gridappsd.process.request.data.powergridmodel'`

`topic = 'goss.gridappsd.process.request.data.powergridmodel'`


## Topic Channels

`/topic/` is used for communication channels where the API call is to broadcast to all subscribers through the GOSS Message Bus, inlcuding other applications, services, FNCS Bridge, etc. 

API calls to the Simulation, services, and active applications use topic channels to communicate and typically need to the specify the Simulation ID, Service ID, and Application ID. The particular topic for such an API call will change between simulations and instances, and so shortcut functions are provided in GridAPPSD-Python library to assist in generating the correct Topic.

In GridAPPSD-Python, it is necessary to specify if a GridAPPS-D Topic is a `/topic/` channel broadcasting to all subscribers:

`topic = "/topic/goss.gridappsd.simulation.input."+simulation_id`

---
# Advantages of Importing the Topics Library

The GridAPPSD-Python Library contains several shortcuts so that you can use the correct topic quickly, without needing to look up the correct string for a particular API call. It also helps make your code more readable.

## Queue Channels

The GridAPPSD-Python Library contains shortcut functions for all of the API calls that use queue channels to communicate with the GridAPPS-D Platform. As a result, a general user would not need to look up these topics. Advanced developers can find more details and shortcuts in the Advanced Concepts section at the end of this tutorial.

## Topic Channels

The GridAPPSD-Python Library contains several functions for creating topics for passing API calls to the GridAPPS-D Platform, applications, and services. A complete list of functions is provided below.

---
# GridAPPSD-Python Topics Functions

Before running any of the sample routines in this tutorial, it is first necessary to establish a connection to the GridAPPS-D Platform so that we can start passing calls to the API. 

Start the GridAPPS-D Platform and start a simulation of any model. Then run the two calls below to connect to the platform and obtain the Simulation ID.

In [ ]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD, utils
gridappsd_conn = GridAPPSD(utils.get_gridappsd_simulation_id, address=utils.get_gridappsd_address(),
          username=utils.get_gridappsd_user(), password=utils.get_gridappsd_pass())

In [ ]:
# Obtain Simulation ID
simulation_id = '12345678'

# simulation_id = utils.get_gridappsd_simulation_id() # Doesn't work yet
# print(simulation_id)

## Subscribe to Simulation Output

This topic is used to communicate with the Simulation API, which is covered in detail in Lesson XX. The Simulation Output Topic is used to subscribe to the simulation output, which applications use to
* Listen to switching actions
* Obtaining equipment measurements
* ___GET FULL LIST___

The GridAPPSD-Python shortcut function for generating the correct topic is

`simulation_output_topic(simulation_id)`

There are two ways to use the function. The first is to call the library function directly. The second is to use it as part of a class definition.

__1) Call the topic function directly__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_output_topic

# Call GridAPPSD-Python Topic Function
topic = simulation_output_topic(simulation_id)

# Print to Notebook Kernel:
print(topic)

__2) Use the topic function in a class definition__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_output_topic

# Define Subscription Class
class MySubscription(object):
    def __init__(self,simulation_id):
        self._subscribe_to_topic = simulation_output_topic(simulation_id)
        
# Define Main Function: 
def _main():
    subscription = MySubscription(simulation_id)
    print(subscription._subscribe_to_topic)
    
# Call Main Function:
_main()

## Publish to Simulation Output

This topic is used to communicate with the Simulation API, which is covered in detail in Lesson XX. The Simulation Input Topic is used to subscribe to the simulation output, which applications use to
* Listen to switching actions
* List

## Request Platform Logs

This topic is used to communicate with the Logging API, which is covered in detail in Lesson XX. The Platform Logs Topic is used to 
* DO SOMETHING

The function used to create a Platform Logs Topic is

`platform_log_topic()`

___NEED TO UNDERSTAND DIFFERENCE BETWEEN PLATFORM.LOG AND SIMULATION.LOG___

---
# Advanced Concepts & Development Tools




We can take a look again at the example from Lesson 2, performing a basic query for the model MRIDs:

In [ ]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD
gridappsd_conn = GridAPPSD("('localhost', 61613)", username='system', password='manager')

# Form JSON query message:
message = '{"requestType": "QUERY_MODEL_NAMES", "resultFormat": "JSON"}'

Original method from Lesson 2 - Not using topics library:

In [ ]:
topic = "goss.gridappsd.process.request.data.powergridmodel"
gridappsd_conn.get_response(topic, message)

Using a pre-formed GridAPPS-D topic:

In [ ]:
from gridappsd import topics as t
gridappsd_conn.get_response(t.REQUEST_POWERGRID_DATA, message)

----------
## Old material (delete when done)

The GridAPPS-D API uses topics to specify the channel on which to communicate.

There are also several topics for the platform, services, applications, and simulations to which one can subscribe:

* _platform_log_topic_ -- Utility method for getting the platform.log base topic

* _service_input_topic_ -- Utility method for getting the input topic for a specific service

* _service_output_topic_ -- Utility method for getting the output topic for a specific service

* _application_input_topic_ -- Utility method for getting the input topic for a specific application

* _application_output_topic_ -- Utility method for getting the output topic for a specific application

* _simulation_input_topic_ -- Gets the topic to write data to for the simulation

* _simulation_output_topic_ -- Gets the topic for subscribing to output from the simulation

* _simulation_log_topic_ -- Topic for the [Logging API](https://gridappsd.readthedocs.io/en/latest/using_gridappsd/index.html#subscribing-to-logs)
